# Use Case for Prompt using footnote format

- Author: [Jinu Cho](https://github.com/jinucho)
- Peer Review: 
- This is a part of [LangChain Open Tutorial](https://github.com/LangChain-OpenTutorial/LangChain-OpenTutorial)

## Overview

This tutorial is an improved prompt version of the **MakeReport-Using-RAG-Websearching-Imagegeneration-Agent**

For more details, please refer to [MakeReport-Using-RAG-Websearching-Imagegeneration-Agent](https://colab.research.google.com/github/LangChain-OpenTutorial/LangChain-OpenTutorial/blob/main/15-Agents/09-MakeReport-Using-RAG-Websearching-Imagegeneration-Agent.ipynb)

---

## Environment Setup

Set up the environment. You may refer to [Environment Setup](https://wikidocs.net/257836) for more details.

**[Note]**
- `langchain-opentutorial` is a package that provides a set of easy-to-use environment setup, useful functions and utilities for tutorials. 
- You can checkout the [`langchain-opentutorial`](https://github.com/LangChain-OpenTutorial/langchain-opentutorial-pypi) for more details.

In [1]:
%%capture --no-stderr
%pip install langchain-opentutorial

In [2]:
# Install required packages
from langchain_opentutorial import package

package.install(
    [
        "langsmith",
        "langchain",
        "langchain_openai",
        "langchain_core",
        "langchain_community",
        "langchain_text_splitters",
        "faiss-cpu",
        "pymupdf",
        "pydantic",
    ],
    verbose=False,
    upgrade=False,
)


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# Set environment variables
from langchain_opentutorial import set_env

set_env(
    {
        "OPENAI_API_KEY": "",
        "LANGCHAIN_API_KEY": "",
        "LANGCHAIN_TRACING_V2": "true",
        "LANGCHAIN_ENDPOINT": "https://api.smith.langchain.com",
        "LANGCHAIN_PROJECT": "Prompt-using-footnote-format",
        "TAVILY_API_KEY": "",
    }
)

Environment variables have been set successfully.


You can alternatively set `OPENAI_API_KEY` in `.env` file and load it. 

[Note] This is not necessary if you've already set `OPENAI_API_KEY` in previous steps.

In [4]:
from dotenv import load_dotenv

load_dotenv(override=True)

True

In [5]:
from langchain_community.tools.tavily_search import TavilySearchResults

# Create an instance of TavilySearchResults with k=6 for retrieving up to 6 search results
search_tool = TavilySearchResults(k=6)

Document Used for Practice  

**Tesla's Revenue Forecast Based on Business Model and Financial Statement Analysis**  

**Author:** Chenhao Fang  
**Institution:** Intelligent Accounting Management Institute, Guangdong University of Finance and Economics  
**Link:** [Tesla's revenue forecast base on business model and financial statement analysis ](https://www.shs-conferences.org/articles/shsconf/pdf/2024/01/shsconf_icdeba2023_02022.pdf)  
**File Name:** shsconf_icdeba2023_02022.pdf

_Please copy the downloaded file to the data folder for practice._  


In [6]:
import os
import requests

# PDF URL
url = "https://www.shs-conferences.org/articles/shsconf/pdf/2024/01/shsconf_icdeba2023_02022.pdf"

# File name to save
file_name = "shsconf_icdeba2023_02022.pdf"

# Custom headers to mimic a browser request
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
    "Referer": "https://www.shs-conferences.org/",
}

# Check if the file already exists
if not os.path.exists(file_name):
    # Send request with headers
    response = requests.get(url, headers=headers, stream=True)
    if response.status_code == 200:
        with open(file_name, "wb") as pdf_file:
            for chunk in response.iter_content(1024):
                pdf_file.write(chunk)
        print(f"PDF has been saved as '{file_name}'.")
    else:
        print(f"Failed to download PDF: {response.status_code}")
else:
    print(f"File '{file_name}' already exists. Skipping download.")

File 'shsconf_icdeba2023_02022.pdf' already exists. Skipping download.


In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader

# Example PDF file path (modify according to your environment)
pdf_file_path = "shsconf_icdeba2023_02022.pdf"

# Load the PDF using PyMuPDFLoader
loader = PyMuPDFLoader(pdf_file_path)

# Split text into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = loader.load_and_split(text_splitter)

# Create FAISS VectorStore
vector = FAISS.from_documents(split_docs, OpenAIEmbeddings())

# Create a retriever from the VectorStore
retriever = vector.as_retriever()

In [8]:
from langchain_core.tools.retriever import create_retriever_tool
from langchain_core.prompts import PromptTemplate

document_prompt = PromptTemplate.from_template(
    "<document>\n"
    "  <content>{page_content}</content>\n"
    "  <page>{page}</page>\n"
    "  <source>{source}</source>\n"
    "</document>"
)

retriever_tool = create_retriever_tool(
    retriever,
    name="pdf_search",
    description="use this tool to search for information in Tesla PDF file",
    document_prompt=document_prompt,
)

In [9]:
from langchain_community.utilities.dalle_image_generator import DallEAPIWrapper
from langchain_core.tools import tool

dalle = DallEAPIWrapper(
    model="dall-e-3",  # DALL·E model version
    size="1024x1024",  # Image size
    quality="standard",  # Image quality
    n=1,  # Number of images to generate
)


@tool
def dalle_tool(query: str) -> str:
    """Use this tool to generate an image from text"""
    return dalle.run(query)

In [10]:
from langchain_community.agent_toolkits import FileManagementToolkit

working_directory = "tmp"

file_tools = FileManagementToolkit(
    root_dir=str(working_directory),
    selected_tools=["write_file", "read_file", "list_directory"],
).get_tools()

In [11]:
tools = file_tools + [
    retriever_tool,  # PDF search (RAG)
    search_tool,  # Web search
    dalle_tool,  # Image generation
]
tools

[WriteFileTool(root_dir='tmp'),
 ReadFileTool(root_dir='tmp'),
 ListDirectoryTool(root_dir='tmp'),
 Tool(name='pdf_search', description='use this tool to search for information in Tesla PDF file', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x144f69120>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x16b638980>, search_kwargs={}), document_prompt=PromptTemplate(input_variables=['page', 'page_content', 'source'], input_types={}, partial_variables={}, template='<document>\n  <content>{page_content}</content>\n  <page>{page}</page>\n  <source>{source}</source>\n</document>'), document_separator='\n\n'), coroutine=functools.partial(<function _aget_relevant_documents at 0x145402fc0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x

In [12]:
from langchain.agents import AgentExecutor, create_tool_calling_agent
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate, load_prompt
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    MessagesPlaceholder,
    SystemMessagePromptTemplate,
)
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_openai import ChatOpenAI

store = {}

# Load the YAML file (MakeReport.yaml)
yaml_prompt = load_prompt("MakeReport.yaml")

# Create a ChatPromptTemplate using the template from the YAML file as a system message,
# and pass along the metadata as defined in the YAML.
prompt = ChatPromptTemplate(
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(
                template=yaml_prompt.template,
                input_variables=yaml_prompt.input_variables,
                partial_variables=yaml_prompt.partial_variables,
                input_types=getattr(yaml_prompt, "input_types", {}),
            )
        ),
        # Include placeholders defined in the YAML file
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        MessagesPlaceholder(variable_name="agent_scratchpad", optional=True),
        # Human message: create a template that only accepts user input
        HumanMessagePromptTemplate(
            prompt=PromptTemplate(template="{input}", input_variables=["input"])
        ),
    ],
    # Pass along the metadata defined in the YAML file
    input_variables=yaml_prompt.input_variables,
    optional_variables=yaml_prompt.optional_variables,
    partial_variables=yaml_prompt.partial_variables,
    input_types=getattr(yaml_prompt, "input_types", {}),
)


llm = ChatOpenAI(model="gpt-4o", temperature=0)

agent = create_tool_calling_agent(llm, tools, prompt)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=False,
    handle_parsing_errors=True,
)


def get_session_history(session_ids):
    if session_ids not in store:
        store[session_ids] = ChatMessageHistory()
    return store[session_ids]


agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [13]:
from langchain_core.agents import AgentAction, AgentStep
from langchain.agents.output_parsers.tools import ToolAgentAction

from typing import Any, Dict, List, Callable
from dataclasses import dataclass


# Callback for tool calls
def tool_callback(tool) -> None:
    print("[Tool Called]")
    print(f"Tool: {tool.get('tool')}")
    if tool_input := tool.get("tool_input"):
        for k, v in tool_input.items():
            print(f"{k}: {v}")
    print(f"Log: {tool.get('log')}")


# Callback for observations
def observation_callback(observation) -> None:
    print("[Observation]")
    print(f"Observation: {observation.get('observation')}")


# Callback for final result
def result_callback(result: str) -> None:
    print("[Final Answer]")
    print(result)


@dataclass
class AgentCallbacks:
    tool_callback: Callable[[Dict[str, Any]], None] = tool_callback
    observation_callback: Callable[[Dict[str, Any]], None] = observation_callback
    result_callback: Callable[[str], None] = result_callback


class AgentStreamParser:
    """
    A class to parse and handle streaming output from the agent.
    """

    def __init__(self, callbacks: AgentCallbacks = AgentCallbacks()):
        """Initialize the AgentStreamParser with custom callbacks."""
        self.callbacks = callbacks
        self.output = None

    def process_agent_steps(self, step: Dict[str, Any]) -> None:
        """Process each step in the agent's output."""
        if "actions" in step:
            self._process_actions(step["actions"])
        elif "steps" in step:
            self._process_observations(step["steps"])
        elif "output" in step:
            self._process_result(step["output"])

    def _process_actions(self, actions: List[Any]) -> None:
        """Handle each tool action detected."""
        for action in actions:
            if isinstance(action, (AgentAction, ToolAgentAction)) and hasattr(
                action, "tool"
            ):
                self._process_tool_call(action)

    def _process_tool_call(self, action: Any) -> None:
        """Handle logic for a single tool call."""
        tool_action = {
            "tool": getattr(action, "tool", None),
            "tool_input": getattr(action, "tool_input", None),
            "log": getattr(action, "log", None),
        }
        self.callbacks.tool_callback(tool_action)

    def _process_observations(self, observations: List[Any]) -> None:
        """Handle the observations produced by the agent."""
        for observation in observations:
            observation_dict = {}
            if isinstance(observation, AgentStep):
                observation_dict["observation"] = getattr(
                    observation, "observation", None
                )
            self.callbacks.observation_callback(observation_dict)

    def _process_result(self, result: str) -> None:
        """Handle the final result from the agent."""
        self.callbacks.result_callback(result)
        self.output = result


# Create the stream parser
agent_stream_parser = AgentStreamParser()

In [14]:
# This cell asks the agent to summarize certain aspects of the Tesla PDF,
# then writes them to 'report.md' and reads the file contents.

query = "Generate a comprehensive 2024-2025 financial and market performance report about Tesla."


result = agent_with_chat_history.stream(
    {"input": query}, config={"configurable": {"session_id": "session_1"}}
)

print("=== Agent Execution Result ===")
for step in result:
    agent_stream_parser.process_agent_steps(step)

=== Agent Execution Result ===
[Tool Called]
Tool: pdf_search
query: Tesla financial information and revenue outlook 2024-2025
Log: 
Invoking: `pdf_search` with `{'query': 'Tesla financial information and revenue outlook 2024-2025'}`



[Observation]
Observation: <document>
  <content>dollars, +18% year-on-year. Assuming a vehicle price of 
$45,000 and $43,000 in 24 and '25, by 2025, the revenue 
from electric vehicle sales will be about $130 billion, +80% 
from '22. All of this is based on an analysis of Tesla's 
existing assets and liabilities and business model. This study 
predicts Tesla's income in the next few years. It is certain 
that Tesla's income will continue to grow in the next few 
years, which may help some investors related to Tesla in 
terms of investment. However, since Tesla discloses less 
economic information to the outside world, and the research 
method of this paper is relatively simple, the information 
that can be provided is limited. In future studies, resear